I'm very happy to challenge new TableData Competition!<br>
So I make very simple EDA.<br>
<br>
This competion has unique rule.
We should check following notebook carefully.<br>
https://www.kaggle.com/dster/nfl-big-data-bowl-official-starter-notebook

NFL Basic rule(English site)<br>
https://operations.nfl.com/the-rules/2019-nfl-rulebook/

NFL Basic rule(Japanese site)<br>
https://nfljapan.com/guide/rule

# Import

In [ ]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, gc, warnings, random, datetime, math

warnings.filterwarnings('ignore')

from kaggle.competitions import nflrush

In [ ]:
# You can only call make_env() once, so don't lose it!
env = nflrush.make_env()

# Setting

In [ ]:
# Configuration
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
SEED = 42
TARGET = 'Yards'

# Read Data

In [ ]:
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

In [ ]:
# You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()

In [ ]:
(test_df, sample_prediction_df) = next(iter_test)

# OverView

### train_df

In [ ]:
train_df.head()

From Official<br>
> Each row in the file corresponds to a single player's involvement in a single play. <br>
> The dataset was intentionally joined (i.e. denormalized) to make the API simple. <br>
> All the columns are contained in one large dataframe which is grouped and provided by PlayId.

For example,following dataframe is 1 play data at same time.

In [ ]:
train_df[train_df["PlayId"] == 20170907000118]

We must do "groupby" by PlayId.

In [ ]:
train_df.groupby("PlayId").first().head()

In the following, I check both original data and grupoby_PlayId data.

In [ ]:
train_df_groupby = train_df.groupby("PlayId").first()

In [ ]:
print(train_df.shape)
print(train_df_groupby.shape)

In [ ]:
train_df.dtypes

In [ ]:
#Missing Value Count
train_df.isnull().sum()

### test_df

In [ ]:
test_df.head()

In [ ]:
test_df.shape

### sample_prediction_df

In [ ]:
sample_prediction_df

In [ ]:
sample_prediction_df.shape

> # train_df, train_df_groupby

In [ ]:
train_df.columns.tolist()

### Predict Value:Yards

We will predict "Yards".<br>
Let's check.

> Yards - the yardage gained on the play (you are predicting this)[](http://)

In [ ]:
def make_histogram(col):
    plt.figure(figsize=(15,5))
    plt.subplot(121);
    plt.title(col+"_histogram"+"_Original")
    sns.distplot(train_df[col].dropna(),kde=False);
    plt.subplot(122);
    plt.title(col+"_histogram"+"_groupby")
    sns.distplot(train_df_groupby[col].dropna(),kde=False);

In [ ]:
def make_describe(col):
    print(f"###############{col} : describe###############")
    print("\n")
    print(f"######Original Data#######")
    print(train_df[col].describe())
    print("\n")
    print(f"######groupby Data#######")
    print(train_df_groupby[col].describe())

In [ ]:
make_histogram("Yards")

In [ ]:
make_describe("Yards")

### Numeric Varaiable

In [ ]:
train_df_num = train_df.select_dtypes(exclude="object")

not_num_value = ["GameId","PlayId","Season","Week","Quarter","Down","NflId","NflIdRusher"] 
#I deal with "Season","Week","Quarter","Down","NflId","NflIdRusher" as categorical value. 
train_df_num = train_df_num.drop(not_num_value,axis=1)

not_num_value.remove("PlayId")
train_df_groupby_num = train_df_groupby.drop(not_num_value,axis=1)

In [ ]:
train_df_num.describe()

In [ ]:
train_df_groupby_num.describe()

Make histogram for Numeric Varaiable

In [ ]:
train_df_num_list = train_df_num.columns.tolist()

for i in train_df_num_list:
    make_histogram(i)

### Categorical Variable

In [ ]:
train_df_cat_list = train_df.select_dtypes("object").columns.tolist()
train_df_cat_list += not_num_value

In [ ]:
for i in train_df_cat_list:
    print(f"###############{i} : Value Counts###############")
    print(train_df[i].value_counts())
    print("\n")

In [ ]:
for i in train_df_cat_list:
    print(f"###############{i} : Value Counts###############")
    print(train_df_groupby[i].value_counts())
    print("\n")

Make histogram for categorical variable which has a few values.

In [ ]:
train_df_cat_list_selected = ['OffenseFormation','PlayDirection','Season','Week','Quarter','Down']

In [ ]:
def make_histogram_cat(col):
    plt.figure(figsize=(15,5))
    plt.subplot(121);
    plt.title(col+"_histogram"+"_Original")
    sns.countplot(x=col, data=train_df);
    plt.subplot(122);
    plt.title(col+"_histogram"+"_groupby")
    sns.countplot(x=col, data=train_df_groupby);

In [ ]:
for i in train_df_cat_list_selected:
    make_histogram_cat(i)

To be continued・・・